# Tf.data.dataset 활용

Represnets a potentially large set of elements

tensorflow 에서 feed-dict 만을 사용해서 데이터를 처리하는 방법은 느리고 권장되는 방법이 아니다. 모델에 데이터를 제대로 공급하려면 입력 파이프라인을 만들어 GPU로 들어올 데이터를 효율적으로 공급해 주어야 한다.  

이를 위해 tensorflow에서는 dataset이라는 built-in-API를 제공하고 있어 위의 작업을 쉽게 처리할 수 있다.  

아래서는 dataset을 사용하여 입력 파이프라인을 직접 만들어보고 모델에 데이터를 효율적으로 공급하는 방법을 알아본다. 

### Dataset을 사용하기 위한 3단계

1. 데이터 불러오기 : 사용하려는 데이터로부터 Dataset 인스턴스를 만든다.
2. Iterator(반복자) 생성 : 생성된 데이터를 사용하여 Iterator 인스턴스를 만들어 Dataset을 iterate한다.
3. 데이터 사용 : 생성된 iterator를 사용해서 모델에 공극ㅂ할 dataset으로 부터 요소를 가져올 수 있다.

(iteration)반복은 스트리밍 방식으로 발생하므로 전체 데이터 세트가 메모리에 맞지 않아도 된다.

### 1. 데이터를 불러오자

일단 dataset안에 넣을 데이터를 불러온다.

### numpy에서 불러오기
numpy 배열이 있고 그걸 tensorflow에 넣는 방법

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
x = np.random.sample((100,2))

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset

<TensorSliceDataset shapes: (2,), types: tf.float64>

In [11]:
for idx,element in enumerate(dataset):
  if idx > 5 :
    break
  print(element)

tf.Tensor([0.58150379 0.92254424], shape=(2,), dtype=float64)
tf.Tensor([0.96896941 0.64480046], shape=(2,), dtype=float64)
tf.Tensor([0.16717264 0.59388047], shape=(2,), dtype=float64)
tf.Tensor([0.9403603  0.04906635], shape=(2,), dtype=float64)
tf.Tensor([0.74319037 0.8906696 ], shape=(2,), dtype=float64)
tf.Tensor([0.18302749 0.90372335], shape=(2,), dtype=float64)


또한 데이터를 우리가 학습시킬때 처럼 데이터 특성(feature)과 라벨(label)로 나누어 사용하는 경우처럼, 한개이상의 numpy 배열을 넣는것도 가능하다.

In [12]:
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))
for idx,element in enumerate(dataset):
  if idx > 5 :
    break
  print(element)

(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.76700221, 0.8580802 ])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.23088477])>)
(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.30859757, 0.80056717])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.15271162])>)
(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.69157562, 0.44435759])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.54916617])>)
(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.09089438, 0.62459401])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.21828884])>)
(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.27987142, 0.39742835])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.32603395])>)
(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.85457713, 0.81044612])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.14107503])>)


### tensor에서 불러오기

tensor를 사용하여 dataset을 초기화 할 수도 있다.

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(tf.random.uniform([100, 2]))

In [18]:
for idx,element in enumerate(dataset):
  if idx > 5 :
    break
  print(element)

tf.Tensor([0.7001499 0.5962088], shape=(2,), dtype=float32)
tf.Tensor([0.4115411  0.02957225], shape=(2,), dtype=float32)
tf.Tensor([0.2660693 0.8786987], shape=(2,), dtype=float32)
tf.Tensor([0.8970568  0.94381356], shape=(2,), dtype=float32)
tf.Tensor([0.63548017 0.77995265], shape=(2,), dtype=float32)
tf.Tensor([0.36507785 0.28371108], shape=(2,), dtype=float32)


### generator에서 불러오기

generator를 사용해서 dataset을 초기화가 간으한데, 데이터의 원소들이 다른 크기를 가지고 있을때 유용, 이런경우 tensor를 만들 때 사용할 데이터의 type과 shape까지 지정해야 한다. 

#### Generator 란?
- generator : iterator를 생성해주는 함수, 함수안에 yield 키워드를 사용
- generator 특징
  - iterable한 순서가 지정됨(모든 generator는 iterator)
  - 느슨하게 평가된다 (순서의 다음 값은 필요에 따라 계산)
  - 함수의 내부 로컬 변수를 통해 내부상태가 유지된다.
  - 무한한 순서가 있는 객체를 모델링 할 수 있다. (명확한 끝이 없는 데이터 스트림)
  - 자연스러운 스트림 처리를 위 파이프라인으로 구성할수 있다.

#### Generator 간단 사용
  - REPL을 실행
  - yield 키워드를 사용해 generator를 만들어보자
  - yield 가 호출되면 암시적으로 return이 호출되며, 한번더 실행되면 실행되었던 'yield' 다음 코드가 실행된다.

In [62]:
def test_generator() :
  yield 1
  yield 2
  yield 3
gen  = test_generator()
type(gen), next(gen), next(gen), next(gen)

(generator, 1, 2, 3)

이렇게 생성한 generator는 iterable한 객체가 되며 for문에 사용가능

In [68]:
sequence = np.array([[[1]],
                     [[2],[3]],
                     [[3,4,5]],
                     [1]])
def generator() :  
  for i in sequence :
    yield i
gen = generator()

for g in gen :
  print(g)

[[1]]
[[2], [3]]
[[3, 4, 5]]
[1]


In [60]:
import itertools

def gen() :
  for i in itertools.count(1) :
    yield(i, [1]*i)


dataset = tf.data.Dataset.from_generator(
     gen,
     (tf.int64, tf.int64),
     (tf.TensorShape([]), tf.TensorShape([None])))

list(dataset.take(4).as_numpy_iterator())

[(1, array([1])),
 (2, array([1, 1])),
 (3, array([1, 1, 1])),
 (4, array([1, 1, 1, 1]))]

### 변형(Transformations)

일단 dataset을 갖고 나면, transformation을 적용하여 데이터를 준비 시킬수 있다.

In [91]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3])
dataset = dataset.map(lambda x : x*2)
list(dataset.as_numpy_iterator())

[2, 4, 6]